In [12]:
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

sections = {
    "CPE106-4_E02_1T2526": "Engr. Dionis Padilla",
    "CPE106L-4_E01_1T2526": "Engr. Erinn Chloe Sanchez"
}

students = [
    "Evan Josh Landong",
    "Patricia Deeone Macainag",
    "Aethan Carlo Tabungar"
]

attendance_log = pd.DataFrame(columns=["timestamp", "date", "time", "section", "professor", "student", "status"])

button_style = {'font_weight': 'bold', 'border': '2px solid #333'}

header_style = """
    <div style='background: linear-gradient(135deg, #2f2f2f 0%, #444444 100%); 
                padding: 25px; 
                border-radius: 8px; 
                margin-bottom: 20px;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
        <h2 style='color: #fff; margin: 0; font-family: Arial, sans-serif;'>
            📋 StampIn
        </h2>
        <p style='color: #ccc; margin: 5px 0 0 0; font-size: 14px;'>
            Attendance Monitoring System | CPE106L-4/E02
        </p>
    </div>
"""

section_dropdown = widgets.Dropdown(
    options=list(sections.keys()),
    description="Section:",
    style={'description_width': '80px'},
    layout=widgets.Layout(width='450px', height='35px')
)

professor_label = widgets.HTML(value="")
student_widgets = []

save_button = widgets.Button(
    description="💾 Save Attendance",
    button_style='',
    layout=widgets.Layout(width='180px', height='40px'),
    style=button_style
)

export_button = widgets.Button(
    description="📤 Export CSV",
    button_style='',
    layout=widgets.Layout(width='180px', height='40px'),
    style=button_style
)

view_log_button = widgets.Button(
    description="📊 View Log",
    button_style='',
    layout=widgets.Layout(width='180px', height='40px'),
    style=button_style
)

output_area = widgets.Output()
status_message = widgets.HTML(value="")

def update_professor_display(change=None):
    selected_section = section_dropdown.value
    prof_name = sections[selected_section]
    professor_label.value = f"""
        <div style='padding: 10px; background: #f0f0f0; border-radius: 4px; margin: 5px 0;'>
            <span style='color: #666; font-size: 13px;'>Professor:</span>
            <span style='color: #2f2f2f; font-weight: bold; margin-left: 10px;'>{prof_name}</span>
        </div>
    """

def build_student_attendance_widgets():
    global student_widgets
    student_widgets = []
    rows = []
    for i, student in enumerate(students):
        bg_color = '#ffffff' if i % 2 == 0 else '#f0f0f0'
        name_label = widgets.HTML(
            value=f"<div style='padding: 8px; background: {bg_color}; border-radius: 4px;'><span style='color: #2f2f2f; font-weight: 500;'>{student}</span></div>",
            layout=widgets.Layout(width='280px')
        )
        status_dropdown = widgets.Dropdown(
            options=["Present", "Absent", "Late", "Excused"],
            value="Present",
            layout=widgets.Layout(width='140px', height='35px')
        )
        student_widgets.append({'name': student, 'widget': status_dropdown})
        rows.append(widgets.HBox([name_label, status_dropdown], layout=widgets.Layout(margin='2px 0')))
    return widgets.VBox(rows, layout=widgets.Layout(border='1px solid #ddd', border_radius='6px', padding='15px', margin='10px 0'))

def show_status_message(message, msg_type="info"):
    colors = {"success": ("#27ae60", "#d4edda"), "error": ("#e74c3c", "#f8d7da"), "info": ("#3498db", "#d1ecf1"), "warning": ("#f39c12", "#fff3cd")}
    color, bg = colors.get(msg_type, colors["info"])
    status_message.value = f"<div style='background: {bg}; color: {color}; padding: 12px 15px; border-radius: 4px; border-left: 4px solid {color}; margin: 10px 0; animation: fadeIn 0.3s; font-weight: 500;'>{message}</div>"

def save_attendance(button):
    global attendance_log
    save_button.description = "⏳ Saving..."
    save_button.disabled = True
    try:
        now = datetime.now()
        timestamp, date, time_str = now.strftime("%Y-%m-%d %H:%M:%S"), now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S")
        selected_section = section_dropdown.value
        professor = sections[selected_section]
        new_records = []
        for student_data in student_widgets:
            new_records.append({'timestamp': timestamp, 'date': date, 'time': time_str, 'section': selected_section, 'professor': professor, 'student': student_data['name'], 'status': student_data['widget'].value})
        attendance_log = pd.concat([attendance_log, pd.DataFrame(new_records)], ignore_index=True)
        show_status_message(f"✓ Attendance saved successfully at {time_str} | {len(new_records)} records", "success")
        with output_area:
            clear_output(wait=True)
            print(f"{'─'*70}\n✓ SAVE CONFIRMATION\n{'─'*70}")
            print(f"Timestamp:  {timestamp}\nSection:    {selected_section}\nProfessor:  {professor}\nRecords:    {len(new_records)}\n{'─'*70}\n")
            display_summary()
    except Exception as e:
        show_status_message(f"✗ Error saving attendance: {e}", "error")
    finally:
        save_button.description = "💾 Save Attendance"
        save_button.disabled = False

def export_to_csv(button):
    if attendance_log.empty:
        show_status_message("⚠ No attendance records to export yet.", "warning")
        return
    export_button.description = "⏳ Exporting..."
    export_button.disabled = True
    try:
        filename = f"StampIn_Attendance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        attendance_log.to_csv(filename, index=False)
        show_status_message(f"✓ Exported {len(attendance_log)} records to {filename}", "success")
        with output_area:
            clear_output(wait=True)
            print(f"{'─'*70}\n✓ EXPORT SUCCESSFUL\n{'─'*70}\nFile:     {filename}\nRecords:  {len(attendance_log)}\nLocation: Current notebook directory\n{'─'*70}")
    except Exception as e:
        show_status_message(f"✗ Export failed: {e}", "error")
    finally:
        export_button.description = "📤 Export CSV"
        export_button.disabled = False

def view_full_log(button):
    with output_area:
        clear_output(wait=True)
        if attendance_log.empty:
            print("📋 No attendance records available yet.\n   Mark attendance and click 'Save Attendance' to begin logging.")
        else:
            print(f"{'═'*80}\n📋 ATTENDANCE LOG\n{'═'*80}\n")
            display(attendance_log.style.set_properties(**{'text-align': 'left','font-size': '12px'}))
            print(f"\n{'═'*80}\n")
            display_summary()

def display_summary():
    if not attendance_log.empty:
        print(f"📊 SUMMARY STATISTICS\n{'─'*80}\n")
        print("▸ Overall Status Distribution:")
        for status, count in attendance_log['status'].value_counts().items():
            print(f"  {status:12s} : {count:3d} ({(count/len(attendance_log))*100:5.1f}%)")
        try:
            latest_timestamp = attendance_log['timestamp'].max()
            latest_records = attendance_log[attendance_log['timestamp']==latest_timestamp]
            print("\n▸ Most Recent Session:")
            print(f"  Time:      {latest_timestamp}")
            print(f"  Section:   {latest_records['section'].iloc[0]}")
            print(f"  Professor: {latest_records['professor'].iloc[0]}")
            print("\n  Status Breakdown:")
            for status, count in latest_records['status'].value_counts().items():
                print(f"  {status:12s} : {count:3d}")
            print(f"\n{'─'*80}")
        except Exception as e:
            print(f"\n  Could not load recent session: {e}")

section_dropdown.observe(update_professor_display, names='value')
save_button.on_click(save_attendance)
export_button.on_click(export_to_csv)
view_log_button.on_click(view_full_log)

def display_interface():
    clear_output(wait=True)
    header = widgets.HTML(value=header_style)
    update_professor_display()
    section_info = widgets.VBox([
        widgets.HTML(value="<div style='margin: 10px 0;'><b style='color: #2f2f2f;'>Class Information</b></div>"),
        section_dropdown, professor_label
    ], layout=widgets.Layout(border='1px solid #ddd', border_radius='6px', padding='15px', margin='10px 0'))
    student_interface_label = widgets.HTML(value="<div style='margin: 20px 0 10px 0;'><b style='color: #2f2f2f;'>Mark Attendance</b></div>")
    student_interface = build_student_attendance_widgets()
    control_panel = widgets.HBox([save_button, export_button, view_log_button], layout=widgets.Layout(margin='15px 0'))
    display(widgets.VBox([header, section_info, student_interface_label, student_interface, control_panel, status_message, output_area], layout=widgets.Layout(padding='20px')))

print("✓ StampIn system loaded successfully!")
print("Run: display_interface() to start")


✓ StampIn system loaded successfully!
Run: display_interface() to start


In [13]:
display_interface()